# Finetune RuGPTs in huggingface
How to finetune RuGPTs models with huggingface. Example for RuGPT3Small. Nfor other models it will take more GPU memory.

This notebook is valid for all RuGPTs models except RuGPT3XL.
## Install env

In [4]:
'''
import getpass
import os

password = getpass.getpass()
command = "sudo rm -S -rf /usr/local/cuda " 
os.popen(command, 'w').write(password+'\n')

command = "sudo ln -S /usr/local/cuda-10.1 /usr/local/cuda" 
os.popen(command, 'w').write(password+'\n')
'''
!sudo rm -rf /usr/local/cuda
!sudo ln -s /usr/local/cuda-10.1 /usr/local/cuda

In [5]:
%%bash
export LD_LIBRARY_PATH=/usr/lib/

In [5]:
!sudo apt-get install clang-9 llvm-9 llvm-9-dev llvm-9-tools -y

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  binfmt-support binutils binutils-common binutils-x86-64-linux-gnu
  lib32gcc-s1 lib32stdc++6 libasan5 libatomic1 libbinutils libc-dev-bin libc6
  libc6-dev libc6-i386 libclang-common-9-dev libclang-cpp9 libcrypt-dev
  libctf-nobfd0 libctf0 libffi-dev libgc1c2 libgcc-9-dev libgomp1 libitm1
  libllvm9 liblsan0 libncurses-dev libobjc-9-dev libobjc4 libomp-9-dev
  libomp5-9 libpfm4 libquadmath0 libstdc++-9-dev libtinfo-dev libtsan0
  libubsan1 libz3-4 libz3-dev linux-libc-dev llvm-9-runtime manpages-dev
  python3-pygments
Suggested packages:
  binutils-doc clang-9-doc glibc-doc ncurses-doc libomp-9-doc libstdc++-9-doc
  llvm-9-doc python-pygments-doc ttf-bitstream-vera
The following NEW packages will be installed:
  binfmt-support binutils binutils-common binutils-x86-64-linux-gnu clang-9
  lib32gcc-s1 lib32stdc++6 libasan5 libatomic1 libbin

In [32]:
'''
import subprocess
import os

os.environ["PATH"] = os.environ["PATH"]+":/usr/local/cuda/bin/" # adding this line
proc = subprocess.Popen("nvcc", stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
(out, _) = proc.communicate()
print(out)
'''

SyntaxError: EOF while scanning triple-quoted string literal (1633289902.py, line 8)

In [10]:
import subprocess


CUDA_version = [s for s in subprocess.check_output(["nvcc", "--version"]).decode("UTF-8").split(", ") if s.startswith("release")][0].split(" ")[-1]
print("CUDA version:", CUDA_version)

if CUDA_version == "10.0":
    torch_version_suffix = "+cu100"
elif CUDA_version == "10.1":
    torch_version_suffix = "+cu101"
elif CUDA_version == "10.2":
    torch_version_suffix = ""
else:
    torch_version_suffix = "+cu110"

CUDA version: 10.1


If code below doesn't work, check your cuda version and installation here https://pytorch.org/get-started/previous-versions/

In [11]:
!pip install torch==1.7.1{torch_version_suffix} torchvision==0.8.2{torch_version_suffix} torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 735.4 MB 8.1 kB/s eta 0:00:01   |▍                               | 8.7 MB 1.8 MB/s eta 0:06:50     |▊                               | 15.8 MB 2.0 MB/s eta 0:06:07     |█                               | 24.9 MB 1.8 MB/s eta 0:06:41     |█▏                              | 26.6 MB 1.7 MB/s eta 0:06:52     |█▍                              | 32.1 MB 1.8 MB/s eta 0:06:31     |██▉                             | 66.2 MB 1.8 MB/s eta 0:06:12     |███▎                            | 75.7 MB 1.3 MB/s eta 0:08:10     |████▍                           | 101.6 MB 1.9 MB/s eta 0:05:28     |█████                           | 116.2 MB 1.9 MB/s eta 0:05:26     |█████                           | 116.8 MB 1.9 MB/s eta 0:05:25     |█████▍                          | 122.8 MB 1.9 MB/s eta 0:05:21     |██████▍                         | 145.9 MB 1.7 MB/s eta 0:05:49     |███████                         | 163.

In [12]:
!pip install transformers==3.5.1

     |████████████████████████████████| 1.3 MB 322 kB/s eta 0:00:01
     |████████████████████████████████| 880 kB 305 kB/s eta 0:00:01
     |████████████████████████████████| 408 kB 229 kB/s eta 0:00:01
     |████████████████████████████████| 172 kB 188 kB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 500 kB 301 kB/s eta 0:00:01
    ERROR: Command errored out with exit status 1:
     command: /home/richelieugvg/anaconda3/bin/python -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-9ieszm7r/sentencepiece_cddb8c5ad2e64f7a93913b1172c09116/setup.py'"'"'; __file__='"'"'/tmp/pip-install-9ieszm7r/sentencepiece_cddb8c5ad2e64f7a93913b1172c09116/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'

In [14]:
!git clone  https://github.com/sberbank-ai/ru-gpts

Cloning into 'ru-gpts'...
remote: Enumerating objects: 686, done.
remote: Counting objects: 100% (244/244), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 686 (delta 189), reused 165 (delta 165), pack-reused 442
Receiving objects: 100% (686/686), 377.90 KiB | 1.25 MiB/s, done.
Resolving deltas: 100% (427/427), done.


In [15]:
mkdir models/

In [28]:
#!pip uninstall transformers -y
!pip install transformers

  Using cached transformers-4.22.1-py3-none-any.whl (4.9 MB)


In [19]:
!pip install tensorboard

     |████████████████████████████████| 5.9 MB 331 kB/s eta 0:00:01
     |████████████████████████████████| 93 kB 301 kB/s eta 0:00:01
     |████████████████████████████████| 781 kB 291 kB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 411 kB/s eta 0:00:01
     |████████████████████████████████| 167 kB 357 kB/s eta 0:00:01
     |████████████████████████████████| 123 kB 318 kB/s eta 0:00:01
     |████████████████████████████████| 4.7 MB 345 kB/s eta 0:00:01
     |████████████████████████████████| 4.9 MB 143 kB/s eta 0:00:01
     |████████████████████████████████| 155 kB 333 kB/s eta 0:00:01
     |████████████████████████████████| 77 kB 396 kB/s eta 0:00:01
     |████████████████████████████████| 151 kB 334 kB/s eta 0:00:01


## Download files

## Train 
The following code download our model and tokenizer from huggingface and finetune model for generating essays.

This took aroung ten minutes and obtain perplexity = 13-16

In [31]:
!export PYTHONPATH=${PYTHONPATH}:/ru-gpts/
!CUDA_VISIBLE_DEVICES=0 python pretrain_transformers.py \
    --output_dir=models/essays \
    --model_type=gpt2 \
    --model_name_or_path=sberbank-ai/rugpt3small_based_on_gpt2 \
    --do_train \
    --train_data_file=train.txt \
    --do_eval \
    --eval_data_file=valid.txt \
    --per_gpu_train_batch_size 1 \
    --gradient_accumulation_steps 1 \
    --num_train_epochs 5 \
    --block_size 2048 \
    --overwrite_output_dir

09/22/2022 18:51:16 - WARNING - __main__ -   Process rank: -1, device: cpu, n_gpu: 0, distributed training: False, 16-bits training: False
Downloading: 100%|██████████████████████████████| 608/608 [00:00<00:00, 875kB/s]
Downloading: 100%|█████████████████████████| 1.71M/1.71M [00:01<00:00, 1.68MB/s]
Downloading: 100%|██████████████████████████| 1.27M/1.27M [00:04<00:00, 316kB/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/richelieugvg/anaconda3/lib/python3.9/site-packages/transformers/models/auto/modeling_auto.py:1062: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
Downloading: 100%|███████████████████████████| 551M/551M [04:46<00:00, 1.92MB/s]
09/22/2022 18:56:18 - INFO 

## Evaluate model
### Generate with generate_transformers.py script.

In [ ]:
!python ru-gpts/generate_transformers.py \
    --model_type=gpt2 \
    --model_name_or_path=models/essays \
    --k=5 \
    --p=0.95 \
    --length=500 \
    --repetition_penalty=5

### Generate mannualy

In [ ]:
import numpy as np
import torch

In [ ]:
np.random.seed(42)
torch.manual_seed(42)

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [ ]:
tok = GPT2Tokenizer.from_pretrained("models/essays")

In [ ]:
model = GPT2LMHeadModel.from_pretrained("models/essays")

In [ ]:
model.cuda()

In [ ]:
text = "вход: А что вы, батенька, делали намедни?\nвыход: "
inpt = tok.encode(text, return_tensors="pt")

In [ ]:
out = model.generate(inpt.cuda(), max_length=500, repetition_penalty=5.0, do_sample=True, top_k=5, top_p=0.95, temperature=1)

In [ ]:
tok.decode(out[0])

'<s>Тема: «Создает человека природа, но развивает и образует его общество». (В.Т. Белинский)\nСочинение: _Эссе №1«Цель общества есть удовлетворение потребностей» — девиз философов всех времен ной поры («Мы должны стремиться к счастью»). И действительно цели разные бывают у обществ – это потребности удовлетворения естественных влечений или же социально-бытовые нужды; удовлетворены ли эти желания достигнуты благодаря деятельности людей - другой вопрос… В обществе господствует биологическая потребность в наслаждении жизнью(Ж.- Жорес), а социальная связана с потреблением материальных благ для существования.</SOCTERN’Я полностью согласна со высказыванием Акакия Львовича Толстого о том,что цель любого государства состоит не только во власти народа над государством. Но главной целью является само государство! Именно оно определяет жизнь страны как стабильную систему правовых отношений между людьми без принуждения властей.(АП РФ). Я считаю что Толстой прав по этому поводу.. Действительно челов